# Taproot: MuSig2 + Tapscript Spends

In this module we'll be making a series of MuSig2 scripts.

We'll then build a MAST of the scripts, and lock up some bitcoin to them. 
Finally we'll practice unlocking the script using a tapscript spend.

## Picking our private keys

We'll need three private keys. Pick three keys (integers) that are greater than 1 and less than `n`,

`n = 115792089237316195423570985008687907852837564279074904382605163141518161494337`

In [2]:
privkeys = [444, 555, 666]

In [3]:
from coincurve import PrivateKey

pubkeys = [ PrivateKey.from_int(x).public_key for x in privkeys ]

In [4]:
[ x.format().hex() for x in pubkeys ]

['039e6d1d11b101055620cbf7a8bfda75ac1fac813a96608f9c59eac485c16b9f1f',
 '03d811d8c4323b43702607c25ade936c0ac2e4a44b2ba51f8320c5179e745a7e29',
 '037f75c66c45a52c35ead5970bbfaafdfba626a6ddceabc14e0f8a8c7d88a5772b']

One thing to know about MuSig2 is that it is NOT a threshold signature scheme.

```
N of N

2 of 2 || 3 of 3

2 of 3 with MuSig2  -> BitGo

Keys: 444, 555, 666

"Aggregate key". 
    444 + 555 -> 999

Three scripts to spend:
444 + 555:     <pubkey999> OP_CHECKSIG
444 + 666:     <pubkey1110> OP_CHECKSIG
555 + 666:     <pubkey1221> OP_CHECKSIG
```

In [15]:
from bip327 import individual_pk, get_xonly_pk, key_agg_and_tweak

pk444 = individual_pk((444).to_bytes(32, 'big'))
pk555 = individual_pk((555).to_bytes(32, 'big'))
pk666 = individual_pk((666).to_bytes(32, 'big'))

pubkey999 = get_xonly_pk(key_agg_and_tweak([pk444, pk555], [], []))
pubkey999.hex()

'92a5bec4caef9f87d22dcbcd61b372a3cb84c42192d1314cb327a02fe1a77d64'

In [12]:
leaf_999 = '20' + pubkey999.hex() + 'ac'
leaf_999

'2092a5bec4caef9f87d22dcbcd61b372a3cb84c42192d1314cb327a02fe1a77d64ac'

In [13]:
!bitcoin-cli -regtest decodescript "$leaf_999"

{
  "asm": "92a5bec4caef9f87d22dcbcd61b372a3cb84c42192d1314cb327a02fe1a77d64 OP_CHECKSIG",
  "desc": "raw(2092a5bec4caef9f87d22dcbcd61b372a3cb84c42192d1314cb327a02fe1a77d64ac)#mas5yvjn",
  "type": "nonstandard",
  "p2sh": "2Mx526JTk4AtyTQDTrtdd6nsL1SGndRwwdM",
  "segwit": {
    "asm": "0 650b8cba98baafa172f4e3b10995e82eb5cfbc3376fcbd8d0f859beb53d5f63b",
    "desc": "addr(bcrt1qv59cew5ch2h6zuh5uwcsn90g966ul0pnwm7tmrg0skd7k5747casvpt6ca)#6f7gpf9c",
    "hex": "0020650b8cba98baafa172f4e3b10995e82eb5cfbc3376fcbd8d0f859beb53d5f63b",
    "address": "bcrt1qv59cew5ch2h6zuh5uwcsn90g966ul0pnwm7tmrg0skd7k5747casvpt6ca",
    "type": "witness_v0_scripthash",
    "p2sh-segwit": "2N8KMAwwwAvBBDKhm7PLMu21aS8R8yCGBM1"
  }
}


In [16]:
pubkey1110 = get_xonly_pk(key_agg_and_tweak([pk444, pk666], [], []))
leaf_1110 = '20' + pubkey1110.hex() + 'ac'

In [17]:
pubkey1221 = get_xonly_pk(key_agg_and_tweak([pk555, pk666], [], []))
leaf_1221 = '20' + pubkey1221.hex() + 'ac'

In [18]:
tree = [ [ leaf_999, leaf_1110 ], leaf_1221 ]

In [25]:
from codes import size_compact_size
from hashlib import sha256

def tag_hash(tag, data_bytes):
    taghash = sha256(tag).digest()
    return sha256(taghash + taghash + data_bytes).digest()

def make_leaf(script_bytes):
    leaf_version = 0xc0
    # leaf version + len script + script
    data = bytes([leaf_version]) + size_compact_size(len(script_bytes)) + script_bytes
    return tag_hash(b'TapLeaf', data)

def taptree_builder(tree):
    if isinstance(tree, str):
        script_bytes = bytes.fromhex(tree)
        leaf_hash = make_leaf(script_bytes)
        print("leaf:", leaf_hash.hex())
        return leaf_hash
    
    # calculate the branch hash. 
    assert len(tree) == 2
    left = taptree_builder(tree[0])
    right = taptree_builder(tree[1])
    
    # we have to order the left + right "alphabetically"
    # thank you arik :)
    if left > right:
        right, left = left, right
    
    branch_hash = tag_hash(b'TapBranch', left + right)
    print("branch:", branch_hash.hex())
    return branch_hash

In [26]:
root = taptree_builder(tree)
root.hex()

leaf: 91def8beb088933ea25ddb1ba3dcd3460d6bc44192bf9eef3582f56e8bde72b4
leaf: 4c37fa27ed43a8887485ad6df68ffa83dc7e6425ba1ff6f679ec2a8631adec1e
branch: 7de9f7a96c98b4e30a2678d59f139a0ab39f17df4d69cb17db055282e444e3eb
leaf: 2225fa3caf3e513ecf704ffadaf4836f87e48770370df229fc260b1e715f0a3d
branch: c01e60f54cf8e47bf0ecbb6fceac7bdb7de13576fe364f0242ea2aa85b35dbf6


'c01e60f54cf8e47bf0ecbb6fceac7bdb7de13576fe364f0242ea2aa85b35dbf6'

In [31]:
from codes import nums_point

internal_pubkey = nums_point()
internal_pubkey.format().hex()

lifting y


'0250929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac0'

In [34]:
from codes import make_tweak_pubkey, make_external_pubkey
pubkey_bytes = internal_pubkey.point()[0].to_bytes(32, 'big')
tweak_pubkey = make_tweak_pubkey(pubkey_bytes, root)
Q = make_external_pubkey(pubkey_bytes, tweak_pubkey)

Q.format().hex()

lifting y


'02a14e12a29723b933e9d55a9493a7599a00a8a4f3af2cb84095e49e1d5d121689'

In [35]:
def make_p2tr(Q):
    Qx_bytes = Q.point()[0].to_bytes(32, 'big')
    witness_version = 0x51
    return bytes([witness_version]) + size_compact_size(len(Qx_bytes)) + Qx_bytes

In [51]:
scriptpubkey = make_p2tr(Q).hex()
scriptpubkey

'5120a14e12a29723b933e9d55a9493a7599a00a8a4f3af2cb84095e49e1d5d121689'

In [42]:
send_to_addr = !bitcoin-cli -regtest decodescript "$scriptpubkey" | jq -r .address
send_to_addr = send_to_addr[0]
send_to_addr

'bcrt1p598p9g5hywun86w4t22f8f6engq23f8n4uktssy4uj0p6hgjz6ys0nghzr'

In [47]:
txid_to_spend = !bitcoin-cli -regtest sendtoaddress "$send_to_addr" 1.0
txid_to_spend = txid_to_spend[0]

In [48]:
txid_to_spend

'971d85f29d9f08062a3f4dfb97be66299a1839abdf5b54568e4abe9e6e079463'

In [49]:
!bitcoin-cli -regtest getrawtransaction "$txid_to_spend" true | jq .vout

[
  {
    "value": 1,
    "n": 0,
    "scriptPubKey": {
      "asm": "1 a14e12a29723b933e9d55a9493a7599a00a8a4f3af2cb84095e49e1d5d121689",
      "desc": "addr(bcrt1p598p9g5hywun86w4t22f8f6engq23f8n4uktssy4uj0p6hgjz6ys0nghzr)#fq4fatm9",
      "hex": "5120a14e12a29723b933e9d55a9493a7599a00a8a4f3af2cb84095e49e1d5d121689",
      "address": "bcrt1p598p9g5hywun86w4t22f8f6engq23f8n4uktssy4uj0p6hgjz6ys0nghzr",
      "type": "witness_v1_taproot"
    }
  },
  {
    "value": 4.24999681,
    "n": 1,
    "scriptPubKey": {
      "asm": "1 e55d853b226bfffc01b2302939bca1183e2c5927fc666db21f139d71aa4884f2",
      "desc": "addr(bcrt1pu4wc2wezd0llcqdjxq5nn09prqlzckf8l3nxmvslzwwhr2jgsneqxkaf6l)#8ft8zfxs",
      "hex": "5120e55d853b226bfffc01b2302939bca1183e2c5927fc666db21f139d71aa4884f2",
      "address": "bcrt1pu4wc2wezd0llcqdjxq5nn09prqlzckf8l3nxmvslzwwhr2jgsneqxkaf6l",
      "type": "witness_v1_taproot"
    }
  }
]


In [64]:
vout = 0

unlock_tx_stuffs = f"""
version: 0200 0000
inputs: 01
    txid: {bytes.fromhex(txid_to_spend)[::-1].hex()}
    vout: {vout.to_bytes(4, 'little').hex()}
    scriptSig: 00
    sequence: ffff ffff
outputs: 01
    amount: {(1 * 10 ** 8 - 400).to_bytes(8, 'little').hex()}
    spk: 22 {scriptpubkey}
locktime: 0000 0000
"""

In [60]:
import re
def cleanup_tx(hmn_read_tx):
    """ Given a block of text, strip out everything except 
        the hex strings
    """
    ret_val = []
    lines = hmn_read_tx.split('\n')
    for line in lines:
        substr = line.split(':')[-1]  # suggested-by @chrisguida + @macaki
        ret_val += re.findall(r'[0-9a-fA-F]{2}', substr)
    return ''.join(ret_val)

In [65]:
unlock_tx = cleanup_tx(unlock_tx_stuffs)
unlock_tx

'02000000016394076e9ebe4a8e56545bdfab39189a2966be97fb4d3f2a06089f9df2851d970000000000ffffffff0170dff50500000000225120a14e12a29723b933e9d55a9493a7599a00a8a4f3af2cb84095e49e1d5d12168900000000'

In [66]:
!bitcoin-cli -regtest decoderawtransaction "$unlock_tx"

{
  "txid": "7285be1d947592f8d6618263f61beb6f30c8cb4b983654c48f8073679db9368d",
  "hash": "7285be1d947592f8d6618263f61beb6f30c8cb4b983654c48f8073679db9368d",
  "version": 2,
  "size": 94,
  "vsize": 94,
  "weight": 376,
  "locktime": 0,
  "vin": [
    {
      "txid": "971d85f29d9f08062a3f4dfb97be66299a1839abdf5b54568e4abe9e6e079463",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 0.99999600,
      "n": 0,
      "scriptPubKey": {
        "asm": "1 a14e12a29723b933e9d55a9493a7599a00a8a4f3af2cb84095e49e1d5d121689",
        "desc": "addr(bcrt1p598p9g5hywun86w4t22f8f6engq23f8n4uktssy4uj0p6hgjz6ys0nghzr)#fq4fatm9",
        "hex": "5120a14e12a29723b933e9d55a9493a7599a00a8a4f3af2cb84095e49e1d5d121689",
        "address": "bcrt1p598p9g5hywun86w4t22f8f6engq23f8n4uktssy4uj0p6hgjz6ys0nghzr",
        "type": "witness_v1_taproot"
      }
    }
  ]
}


In [87]:
from codes import parse_compact_size, parse_input_bytes, parse_output_bytes

def parse_witness_bytes(data):
    count, size = parse_compact_size(data)
    ptr = size
    witnesses = []
    for _ in range(0, count):
        witlen, size = parse_compact_size(data[ptr:])
        ptr += size
        witnesses.append(data[ptr:ptr+witlen])
        ptr += witlen

    return witnesses, ptr

def parse_tx_bytes_mine(tx_hex):
  tx_bytes = bytes.fromhex(tx_hex)
  tx = {}
  ptr = 0
  tx['version'] = tx_bytes[0:4]
  ptr += 4

  if tx_bytes[ptr] == 0x00:
    assert tx_bytes[ptr+1] == 0x01
    tx['marker_flag'] = bytes([0x00, 0x01])
    ptr += 2

  count, size = parse_compact_size(tx_bytes[ptr:])
  ptr += size 
  tx['inputs'] = []
  for _ in range(0, count):
    inputx, size = parse_input_bytes(tx_bytes[ptr:])
    ptr += size
    tx['inputs'].append(inputx)
 
  count, size = parse_compact_size(tx_bytes[ptr:])
  ptr += size
  tx['outputs'] = []
  for _ in range(0, count):
    outputx, size = parse_output_bytes(tx_bytes[ptr:])
    ptr += size
    tx['outputs'].append(outputx)

  if 'marker_flag' in tx:
    tx['witnesses'] = []
    for _ in range(0, len(tx['inputs'])):
        witness, size = parse_witness_bytes(tx_bytes[ptr:])
        ptr += size
        tx['witnesses'].append(witness)

  tx['locktime'] = tx_bytes[ptr:]
  return tx

In [91]:
from hashlib import sha256
from codes import size_compact_size

def sigmsg_default(tx_hex, input_index, annex_bytes, amounts_bytes, scriptpubkeys_bytes, ext_flag):
    txdata = parse_tx_bytes_mine(tx_hex)
    
    result = b''
    result += bytes([0x00]) # sighash_flag
    result += txdata['version']
    result += txdata['locktime']

    all_input_outpoints = b''
    for inp in txdata['inputs']:
        # txid || vout
        all_input_outpoints += inp['txid'] + inp['vout']    
    sha_prevouts = sha256(all_input_outpoints).digest()
    result += sha_prevouts
    
    sha_amounts = sha256(b''.join(amounts_bytes)).digest()        
    result += sha_amounts
    
    spks = b''
    for spk in scriptpubkeys_bytes:
        spks += size_compact_size(len(spk)) + spk
    sha_scriptpubkeys = sha256(spks).digest()
    result += sha_scriptpubkeys
    
    all_sequences = [ i['sequence'] for i in txdata['inputs'] ]
    sha_sequences = sha256(b''.join(all_sequences)).digest()
    result += sha_sequences
    
    all_outputs = b''
    for o in txdata['outputs']:
        # amount || compact_size(scriptpubkey) || scriptpubkey
        all_outputs += o['amount']
        all_outputs += size_compact_size(len(o['scriptPubKey']))
        all_outputs += o['scriptPubKey']
    sha_outputs = sha256(all_outputs).digest()
    result += sha_outputs
    
    # data about this input
    annex_present = 1 if annex_bytes else 0
    spend_type = 2 * ext_flag + annex_present
    result += (spend_type).to_bytes(1, 'little')
    result += (input_index).to_bytes(4, 'little')
    
    if (annex_bytes):
        size_bytes = size_compact_size(len(annex_bytes))
        sha_annex = sha256(size_bytes + annex_bytes).digest()
        result += sha_annex
    
    # data about this output
    # noop for SIGHASH_DEFAULT
    
    assert len(result) <= 206
    return result

In [92]:
from codes import parse_tx_bytes
prev_tx = !bitcoin-cli -regtest getrawtransaction "$txid_to_spend"
spent_from_tx = parse_tx_bytes_mine(prev_tx[0])

In [93]:
tapleaf_hash = make_leaf(bytes.fromhex(leaf_999))
key_version = bytes([0x00])
codesep_position = bytes([0xff, 0xff, 0xff, 0xff])

extension_data = tapleaf_hash + key_version + codesep_position
extension_data.hex()

'91def8beb088933ea25ddb1ba3dcd3460d6bc44192bf9eef3582f56e8bde72b400ffffffff'

In [96]:
amounts_bytes = [spent_from_tx['outputs'][vout]['amount']]
print("amounts:", [x.hex() for x in amounts_bytes])

scriptpubkeys_bytes = [spent_from_tx['outputs'][vout]['scriptPubKey']]
print("scriptpubkeys:", [x.hex() for x in scriptpubkeys_bytes], "\n")

ext_flag = 1 if extension_data else 0
annex_bytes = None
input_index = 0

sigmsg = sigmsg_default(unlock_tx, input_index, annex_bytes, amounts_bytes, scriptpubkeys_bytes, ext_flag)

# add extension data to end of what's passed into tag_hash
sighash = tag_hash(b'TapSighash', bytes([0x00]) + sigmsg + extension_data)
sighash.hex()

amounts: ['00e1f50500000000']
scriptpubkeys: ['5120a14e12a29723b933e9d55a9493a7599a00a8a4f3af2cb84095e49e1d5d121689'] 



'74289212d081d27c8b6f0b2ca617ff65ebd293d46f96e4ea699c9b3a61aad3a2'

In [124]:
from bip327 import nonce_gen, nonce_agg, SessionContext, sign, partial_sig_agg

privkey444 = (444).to_bytes(32, 'big')
privkey555 = (555).to_bytes(32, 'big')

sec444, pub444 = nonce_gen(privkey444, pk444, pubkey999, sighash, b'')
sec555, pub555 = nonce_gen(privkey555, pk555, pubkey999, sighash, b'')
pubnonces = [pub444, pub555]
aggnonce = nonce_agg(pubnonces)

In [125]:
sesh = SessionContext(aggnonce, [pk444, pk555], [], [], sighash)

partial_sig444 = sign(sec444, privkey444, sesh)
partial_sig555 = sign(sec555, privkey555, sesh)
sig = partial_sig_agg([partial_sig444, partial_sig555], sesh)

In [126]:
leaf_script = bytes.fromhex(leaf_999)

In [127]:
def control_block_version_byte(Q):
    val = 0xc0
    if Q.point()[1] % 2 != 0:
        val += 1
    return bytes([val])

In [128]:
def build_proof_of_inclusion(tree, script):
    if isinstance(tree, str):
        script_bytes = bytes.fromhex(tree)
        leaf_hash = make_leaf(script_bytes)
        return leaf_hash, [], script == tree
    
    # calculate the branch hash. 
    assert len(tree) == 2
    left_hash, left_proof, left_target = build_proof_of_inclusion(tree[0], script)
    right_hash, right_proof, right_target = build_proof_of_inclusion(tree[1], script)
    
    proof = []
    if left_target:
        left_proof.append(right_hash)
        proof = left_proof
    elif right_target:
        right_proof.append(left_hash)
        proof = right_proof
        
    # we have to order the left + right "alphabetically"
    # thank you arik :)
    if left_hash > right_hash:
        right_hash, left_hash = left_hash, right_hash
    branch_hash = tag_hash(b'TapBranch', left_hash + right_hash)
    return branch_hash, proof, left_target or right_target

In [129]:
_, proof_of_inclusion, _ = build_proof_of_inclusion(tree, leaf_999)
control_block = control_block_version_byte(Q) + internal_pubkey.format()[1:] + b''.join(proof_of_inclusion)

In [130]:
witnesses = [
    sig,
    leaf_script,
    control_block
]

In [132]:
def build_witness_str(wits):
    wit_str = size_compact_size(len(wits)).hex()
    for witness in wits:
        wit_str += size_compact_size(len(witness)).hex() + witness.hex()
    return wit_str

In [134]:
signed_tx_stuffs = f"""
version: 0200 0000
marker + flag: 0001
inputs: 01
    txid: {bytes.fromhex(txid_to_spend)[::-1].hex()}
    vout: {vout.to_bytes(4, 'little').hex()}
    scriptSig: 00
    sequence: ffff ffff
outputs: 01
    amount: {(1 * 10 ** 8 - 400).to_bytes(8, 'little').hex()}
    spk: 22 {scriptpubkey}
witnesses:
    {build_witness_str(witnesses)}
locktime: 0000 0000
"""

signed_tx = cleanup_tx(signed_tx_stuffs)

In [135]:
!bitcoin-cli -regtest decoderawtransaction "$signed_tx"

{
  "txid": "7285be1d947592f8d6618263f61beb6f30c8cb4b983654c48f8073679db9368d",
  "hash": "485fdcff2c17fe40f0bcd2b5a019f3b738d5f2ae81159d88f7451d1b41d4620e",
  "version": 2,
  "size": 295,
  "vsize": 145,
  "weight": 577,
  "locktime": 0,
  "vin": [
    {
      "txid": "971d85f29d9f08062a3f4dfb97be66299a1839abdf5b54568e4abe9e6e079463",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "5dbdaa357bf4db54b5feed3039123c2229bdf3c7e0e38ce3d10d550d27a7a86abe6a8fd3652810c5f5208771ff4d1c15e8b3314c69198915dbadd2db8ac61ebe",
        "2092a5bec4caef9f87d22dcbcd61b372a3cb84c42192d1314cb327a02fe1a77d64ac",
        "c050929b74c1a04954b78b4b6035e97a5e078a5a0f28ec96d547bfee9ace803ac04c37fa27ed43a8887485ad6df68ffa83dc7e6425ba1ff6f679ec2a8631adec1e2225fa3caf3e513ecf704ffadaf4836f87e48770370df229fc260b1e715f0a3d"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 0.99999600,
      "n": 0,
      "scriptPubKey": 

In [136]:
!bitcoin-cli -regtest testmempoolaccept '["'"$signed_tx"'"]'

[
  {
    "txid": "7285be1d947592f8d6618263f61beb6f30c8cb4b983654c48f8073679db9368d",
    "wtxid": "485fdcff2c17fe40f0bcd2b5a019f3b738d5f2ae81159d88f7451d1b41d4620e",
    "allowed": true,
    "vsize": 145,
    "fees": {
      "base": 0.00000400
    }
  }
]
